In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6076/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [5]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2_sup")
        # self.discard_below_average(sort_by="mse_sup")
        # self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results_04.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_45_9_18,0.944778,0.860658,0.990944,0.883406,0.948092,0.078380,0.197775,0.013565,0.196815,0.105190,1.198832,0.279964,1.004194,0.291883,685.092375,1099.510156,"Hidden Size=[21, 12], regularizer=0.02, learni..."
1,model_45_9_17,0.944476,0.860653,0.991349,0.884785,0.948816,0.078809,0.197783,0.012958,0.194487,0.103723,1.202743,0.280729,1.004217,0.292681,685.081458,1099.499238,"Hidden Size=[21, 12], regularizer=0.02, learni..."
2,model_45_9_19,0.945064,0.860647,0.990529,0.882017,0.947360,0.077974,0.197791,0.014187,0.199159,0.106673,1.195002,0.279237,1.004172,0.291125,685.102771,1099.520552,"Hidden Size=[21, 12], regularizer=0.02, learni..."
3,model_45_9_16,0.944156,0.860631,0.991744,0.886153,0.949532,0.079262,0.197813,0.012367,0.192179,0.102273,1.206741,0.281535,1.004241,0.293520,685.069997,1099.487778,"Hidden Size=[21, 12], regularizer=0.02, learni..."
4,model_45_9_20,0.945335,0.860622,0.990106,0.880620,0.946622,0.077589,0.197828,0.014821,0.201518,0.108169,1.191262,0.278548,1.004152,0.290406,685.112666,1099.530447,"Hidden Size=[21, 12], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,model_26_8_0,0.995033,0.823923,0.972523,0.680185,0.969362,0.007051,0.249915,0.037347,0.004838,0.021093,0.157470,0.083968,1.000213,0.087543,1175.909278,1886.513884,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
1625,model_31_8_0,0.965319,0.823911,0.932030,0.958787,0.944003,0.049225,0.249932,0.046439,0.021743,0.034091,0.475499,0.221867,1.001489,0.231313,1172.022701,1882.627306,"Hidden Size=[26, 18], regularizer=0.5, learnin..."
1626,model_36_9_0,0.998567,0.823905,0.986490,0.997738,0.996643,0.002033,0.249942,0.003872,0.001596,0.002734,0.102666,0.045091,1.000042,0.047011,1682.396256,2700.157570,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
1627,model_48_2_0,0.969335,0.823892,0.964529,0.858886,0.953937,0.043524,0.249959,0.039853,0.230307,0.135080,0.517932,0.208624,1.002329,0.217505,686.268892,1100.686672,"Hidden Size=[21, 12], regularizer=0.5, learnin..."
